<span style="font-size:30pt;font-weight:bold">Home Credit Python Scoring Workflow v.0.8.2</font>

**Copyright:**

© 2017-2020, Pavel Sůva, Marek Teller, Martin Kotek, Jan Zeller, Marek Mukenšnabl, Kirill Odintsov, Jan Hynek, Elena Kuchina, Lubor Pacák, Naďa Horká, Kamil Yazigee and Home Credit & Finance Bank Limited Liability Company, Moscow, Russia – all rights reserved

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the [License](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

For list of contributors see [Gitlab page](https://git.homecredit.net/risk/python-scoring-workflow) 

# Import packages

## List of packages
- `time`, datetime - ability to get current time for logs
- `math` - basic mathematical functions (as logarithm etc.))
- `random` - generate random selection from probability distributions
- `NumPy` - for scientific, mathematical, numerical calculations
- `Scipy` - for clustering and correlation calculations
- `Pandas` - for efficient work with large data structures (you need pandas **version 0.25.3**)
- `cx_Oracle` and sqlalchemy - for loading data from Oracle database (DWH etc.)
- `scikit`-learn - all important machine learning (and statistical) algorithms used for training the models
- `matplotlib` - for plotting the charts
- `seaborn` - for statistical visualisations
- `os` - for setting output paths for generated image files
- `pickle` - to save models to external files
- `tqdm` - intelligent progress bar (**version 4.40.0 or higher**)
- `tkinter` - for interactive Interactions tool GUI
- `xgboost` - gradient boosting used for feature selection before regression
- `lightgbm shap hyperopt` - faster implementations of gradient boosting

**If any of these packages is missing, you have to install it from the Anaconda prompt using command *conda install packagename* where *packagename* is the name of the installed package.**

There is another package called *scoring*, which is distributed along with this workflow. **The folder *scoring* must be located in the same folder as this workflow for the package to be loaded correctly.** Alternatively, you can locate it somewhere else and then use *sys.path.insert()* to map this location.

## Other important prerequisites

For the grouping some **extensions for Jupyter must be installed and enabled before Jupyter is started and the notebook is loaded**. These extensions are Javascripts running in the browser, so it is necessary to have a compatibile browser. Generally, Chrome is OK, Internet Explorer 11 is NOT OK. To install the extensions, run this in your Anaconda prompt:

- `conda install ipywidgets`
- `jupyter nbextension enable --py --sys-prefix widgetsnbextension`
- `jupyter nbextension enable --py --sys-prefix qgrid`

To be able to connect to Oracle database (to get the data directly from your DWH) you need a compatibile Oracle driver to be installed on your computer. **With 64-bit Python, you need to have 64-bit Oracle driver installed.** Before you install the driver, you need to have Java 8 JDK (JRE is not enough) installed on your computer.

In [ ]:
import time
import datetime
import operator
import math
import random
import numpy as np
import pandas as pd

# import cx_Oracle
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import pickle
import gc
import json
# check your tqdm version if import fails
from tqdm.notebook import tqdm

# import tkinter

import sys

sys.path.insert(0, "..")
import scoring

Set general technical parameters and paths.

In [ ]:
sns.set()
%matplotlib inline
%config InlineBackend.close_figures=True
from IPython.display import display, Markdown, HTML
pd.options.display.max_columns = None
pd.options.display.max_rows = 15
output_folder = 'documentation'

if not os.path.exists(output_folder): os.makedirs(output_folder)
if not os.path.exists(output_folder+'/performance'): os.makedirs(output_folder+'/performance')
if not os.path.exists(output_folder+'/predictors'): os.makedirs(output_folder+'/predictors')
if not os.path.exists(output_folder+'/stability'): os.makedirs(output_folder+'/stability')
if not os.path.exists(output_folder+'/stability_short'): os.makedirs(output_folder+'/stability_short')
if not os.path.exists(output_folder+'/analysis'): os.makedirs(output_folder+'/analysis')
if not os.path.exists(output_folder+'/model'): os.makedirs(output_folder+'/model')
if not os.path.exists(output_folder+'/nan_share'): os.makedirs(output_folder+'/nan_share')
scoring.check_version('0.8.2', list_versions=False)

# Input data

## Import data
Importing data from a CSV file. It is important to set the following parameters:

encoding: usually 'utf-8' or windows-xxxx on Windows machines, where xxxx is 1250 for Central Europe, 1251 for Cyrilic etc.
sep: separator of columns in the file
decimal: decimal dot or coma
index_col: which columns is used as index - should be the unique credit case identifier

**Defining NA values:** In different datasets, there can be different values to be considered *N/A*. By default, we set only blank fields to be considered *N/A*, however you might want to change it and add values like *'NA'*, *'NAN'*, *'null'* to be also considered *N/A*. User parameter `na_values` for this.

In [ ]:
from scoring import db

data = db.read_csv(
    r"demo_data\ExampleData8.CSV",
    sep=",",
    decimal=".",
    optimize_types=True,
    encoding="utf-8",
    index_col="ID",
    low_memory=False,
    keep_default_na=False,
    na_values=[""],
)
print("Data loaded on", datetime.datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d %H:%M:%S"))

The data need to have **index column which has unique value per each row**. If not, it can cause some problems later. Run this to deal with such rows:

In [ ]:
# data.info()

In [ ]:
# Option 1: remove rows with duplicated index
# data=data[~data.index.duplicated(keep='first')]

# #Option 2: reset index
# data['INDEX_ORIGINAL'] = data.index
# data.reset_index(inplace=True)

Optionally the data can be loaded also from a database. The function read_sql uses cache, so the data don't have to be downloaded from the database repeatedly. The cache will be located in a new folder called **db_cache**.

In [ ]:
# from sqlalchemy import create_engine
# engine = create_engine('oracle://PAVELS[GP_HQ_RISK]:xxx@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=DBDWHRU.HOMECREDIT.RU)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=DWHRU)))', echo=False)

In [ ]:
# from scoring.db import read_sql
# ru_data = read_sql('select * from owner_dwh.f_application_tt where rownum<11',engine, index_col = 'sk_application')
# print('Data loaded on',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

If you need to download data from the database again (and not from cache), use the parameter refresh:

In [ ]:
# from scoring.db import read_sql
# data = read_sql('select * from owner_dwh.f_application_base_tt where rownum=1',engine, index_col = 'skp_application',refresh=True)
# print('Data loaded on',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
print("Number of rows:", data.shape[0])
print("Number of columns:", data.shape[1])

## Metadata definitions
Assigning ID column, target column, time column and month column. The month column don't have to exist in the dataset, it will be created later in this workflow.

In [ ]:
### THESE COLUMNS MUST BE INCLUDED IN THE DATA SET ###
# name of the target column
col_target = "DEF"
# name of the time column
col_time = "TIME"

### THESE COLUMNS DON'T HAVE TO BE INCLUDED IN THE DATA SET AND ARE CREATED AUTOMATICALLY LATER ###
# name of the base column
col_base = "BASE"
# name of the month column
col_month = "MONTH"
# name of the day column
col_day = "DAY"
# name of the weight column - CURRENTLY COMMENTED OUT BECAUSE OF REASONS MENTIONED LATER
col_weight = "WEIGHT"
# name of the reject column - only use if exists in your data, used for reject inference analysis
col_reject = "REJECTED"

Inicialize doctools object for documentation generation.

In [ ]:
from scoring import doctools

documentation = doctools.ProjectParameters()

Set a list of targets with their respective bases, time variable and rowid variable for plotting.

In [ ]:
documentation.targets = [(col_target, col_base)]
documentation.time_variable = col_month
documentation.rowid_variable = "ID"

In [ ]:
data[col_target] = data[col_target].astype(np.float)

If you don't have base column in your data set, the following code adds it (based on if target is filled).

In [ ]:
if col_base not in data:
    data[col_base] = 0
    data.loc[data[col_target] == 0, col_base] = 1
    data.loc[data[col_target] == 1, col_base] = 1
    print("Column", col_base, "added/modified. Number of columns:", data.shape[1])
else:
    print("Column", col_base, "already exists.")

If you don't have weight column in your data set, the following code adds it, with value = 1 for each row. **The weights are currently supported by Data Exploration, Interactive Grouping and Model Selection (L1 regression and Stepwise) classes, but not by all functions in the workflow. This is why they are commented out now (can be uncommented by the user).**

In [ ]:
# if col_weight not in data:
#    data[col_weight] = 1
#    print('Column',col_weight,'added/modified. Number of columns:',data.shape[1])
# else:
#    print('Column',col_weight,'already exists.')

Create the month and day column from the time column is doing the following
- take the time column and tell in which format the time is saved in - **you need to specify this in variable *dtime_input_format*** (see https://docs.python.org/3/library/time.html#time.strftime for reference)
- strip the format just to year, month, day string
- convert the string to number
- the new column will be added to the dataset as day
- truncate this column to just year and month and add it to dataset as month

In [ ]:
dtime_input_format = "%Y-%m-%d %H:%M:%S"

In [ ]:
data[col_day] = pd.to_numeric(pd.to_datetime(data[col_time], format=dtime_input_format, cache=False).dt.strftime("%Y%m%d"))
data[col_month] = data[col_day].apply(lambda x: math.trunc(x / 100))
print("Columns", col_day, "and", col_month, "added/modified. Number of columns:", data.shape[1])

In [ ]:
data.head(5)

Load the predictors list from a csv file. The csv should have just one column, without any header, containing the name of the variables that should be used as predictors.

Support for boolean predictors in **not** currently implemented. Convert boolean predictors to object to use them.

`s = s.apply(lambda value: str(value) if not np.isnan(value) else value).astype('object')`

In [ ]:
from scoring.data_manipulation import split_predictors_bytype

cols_pred = list(pd.read_csv(r'demo_data\ExamplePredList8.CSV', sep = ',', decimal = '.', 
                   encoding = 'windows-1251', low_memory = False, header = None)[0])

cols_pred, cols_pred_num, cols_pred_cat = split_predictors_bytype(data,
                                                                  pred_list=cols_pred,
                                                                  non_pred_list= [],
                                                                  optimize_types=True,
                                                                  convert_bool2int=True)

**Please check if all predictors were categorized correctly.** 
`Category` dtype is now used for categorical columns for memory efficiency. This means it will not be editable as *string*. If you need to edit values of a categorical column convert it to *string* using this syntax:

`data['Column name'] = data['Column name'].astype(str)`

## Data exploration

In [ ]:
descrip = data.describe(include="all").transpose()
pd.options.display.max_rows = 1000
display(descrip.fillna(""))
pd.options.display.max_rows = 15

**explore_numerical** and **explore_categorical** functions give graphical data exploratory analyses. They can also output into html files. You just need to specify the folder for output.

If you want the detailed legacy HTML output (v0.4.3) comment-out the cell below

These functions analyze only the part of data where target is not null even if it is not explicitly specified.

In [ ]:
# from scoring.data_exploration import explore_categorical, explore_numerical, join_explorations

# explored_columns = list()
# for name, column in tqdm(data[cols_pred].iteritems(), total=len(cols_pred), leave=False):
#     if name in cols_pred_num[:]:
#         if (column.count() > 0) and (column.max() != column.min()):
#             explore_numerical(column, data[col_target], weightCol=None, htmlOut=True, ntbOut=False, outFolder="exp")
#             explored_columns.append(name)
#     if name in cols_pred_cat[:]:
#         if (column.count() > 0) and (len(set(column.unique()) - {np.nan}) > 1):
#             explore_categorical(column, data[col_target], weightCol=None, htmlOut=True, ntbOut=False, outFolder="exp")
#             explored_columns.append(name)

# # comment out the line below if you didn't generate html files in this cell
# join_explorations(explored_columns, filename="_exploration.html", outFolder="exp", weighted=False)

**explore_df** function creates a simple text report about the important variable. The report can be then printed either to the screen or to a file.

In the following code, only such part of data that has col_base = 1 is analyzed. You can remove the condition if you wish.

In [ ]:
from scoring.data_exploration import explore_df

st = explore_df(data[data[col_base] == 1], col_month, col_target, cols_pred)
print(st, file=open("data_exp.txt", "w", encoding="utf-8"))
# print(st)

**Default rate in time**: Simple visualisation of observation count and default rate in time

In [ ]:
observable_mask = data[col_base] == 1

In [ ]:
documentation.sample_dict = {
    "Observable": observable_mask}

In [ ]:
documentation.PlotDataset(
    data,
    sample="Observable",
    target=col_target,
#     segment_col="Categorical_5",
    output_folder=os.path.join(output_folder, 'analysis'),
    
)

**NaN share by month** for each variable in dataset:

In [ ]:
from scoring.data_exploration import nan_share_development

nan_table = nan_share_development(
    data[cols_pred + [col_month]],
    col_month,
    make_images=True,
    show_images=False,
    output_path=output_folder + "/nan_share/",
)
display(nan_table.replace({0:""}))

## Data split

- Split data into five parts (in time training, in time validation, in time test, out of time, historical out of time)
- Adds a new column indicating to which part the observations belong
- The *splitting_points* (first date of train and first date of out of time sample) can be adjusted (there can be any number of such splitting points) - it should correspond to values of column specified by *time_column* parameter
- For each time split, you can create multiple random splits (i.e. train/valid/test), the ratio of sizes of these splits is set by parameter *sample_sizes*
- The random splits can be stratified by multiple variables, which are specified in a list - argument to *stratify_by_columns* parameter
- Set the random seed so the results are replicable

**Before you run data split, make sure that index in your dataset in unique!** If not, you need to create new unique index.

In [ ]:
# #saving original index to a new column and resetting it
# data['INDEX_ORIGINAL'] = data.index
# data.reset_index(inplace=True)

In [ ]:
from scoring.data_manipulation import data_sample_time_split

col_datatype = "data_type"

data[col_datatype] = data_sample_time_split(
    data,
    time_column=col_month,
    splitting_points=[201702, 201707],
    sample_sizes=[[1], [0.4, 0.3, 0.3], [1]],
    sample_names=[["hoot"], ["train", "valid", "test"], ["oot"]],
    stratify_by_columns=[col_month, col_target],
    random_seed=1234,
)

Masks: boolean vectors corresponding to rows in the datasets. True if an row is observable and its data type belongs to given sample.

In [ ]:
train_mask = (data[col_datatype] == "train") & (data[col_base] == 1)
valid_mask = (data[col_datatype] == "valid") & (data[col_base] == 1)
test_mask = (data[col_datatype] == "test") & (data[col_base] == 1)
oot_mask = (data[col_datatype] == "oot") & (data[col_base] == 1)
hoot_mask = (data[col_datatype] == "hoot") & (data[col_base] == 1)

In [ ]:
documentation.sample_dict.update(
    {"HOOT": hoot_mask, "Train": train_mask, "Valid": valid_mask, "Test": test_mask, "OOT": oot_mask}
)

Add masks to _documentation_ object.

Data summary (number of defaults, number in base, number of observations, default rate) by month and by sample

In [ ]:
# data_summary = data.groupby([col_month, col_datatype]).aggregate({col_target: "sum", col_base: ["sum", "count"]})
# data_summary.columns = [col_target, col_base, "Rows"]
# data_summary[col_target + " rate"] = data_summary[col_target] / data_summary[col_base]
# # display(data_summary)

# data_summary = data_summary.reset_index(level=col_datatype).pivot(columns=col_datatype)
# display(data_summary.round(3).fillna(""))
# data_summary.to_csv(output_folder + "/analysis/summary.csv")

## Metadata export

Metadata from data preparation can be reused for example in gradient boosting workflow

In [ ]:
metadata_export = {
        "col_time": col_time,
        "col_month": col_month,
        "col_day": col_day,
        "col_target": col_target,
        "col_base": col_base,
        "col_weight": col_weight,
        "col_reject": col_reject,
        "col_datatype": col_datatype,
        "col_id": data.index.name,
                  }

pd.DataFrame.from_dict(metadata_export, orient='index').to_csv(output_folder + "/model/metadata.csv", header=None)

# Derived variables

## Date differences

Function *datetime_difference()* takes name of dataset and names of two variables with datetime strings (the format of those strings can be also specified in parameters) and caluclates their difference in specified time units. We then append names of these difference columns to the list of numerical predictors.

First, **please cast your date variables as datetimes.**

In [ ]:
data["DateVariable_1"] = pd.to_datetime(data["DateVariable_1"])
data["DateVariable_2"] = pd.to_datetime(data["DateVariable_2"], format="%Y-%m-%d %H:%M:%S")

It is handful to check for given variables whether they contain values over current date.  
This might indicate so-called **Y2K errors**. 
When date is in format such as 31/10/68, such date will get interpreted as 31st of October, 2068.

If you encounter such variables, there exist function `fix_y2k_errors`.  
You are advised to find the root issue of such dates, though.

In [ ]:
print(f"Maximal date in 'DateVariable_1' is {max(data['DateVariable_1'])}")
print(f"Maximal date in 'DateVariable_2' is {max(data['DateVariable_2'])}")

# from scoring.date_tools import fix_y2k_errors
# data['DateVariable_1'] = fix_y2k_errors(data['DateVariable_1'], years=18)
# data['DateVariable_2'] = fix_y2k_errors(data['DateVariable_2'], reference_date='2000-01-01')

When you are sure that everything is correct, you are free to proceed with calculating datetime differences.

In [ ]:
from scoring.date_tools import datetime_difference

data["AGE"] = datetime_difference(pd.to_datetime(data[col_time]), data["DateVariable_1"], unit="years", rounding=None)
data["NEW_TIME_VARIABLE"] = datetime_difference(
    pd.to_datetime(data[col_time]), data["DateVariable_2"], unit="months", rounding="floor"
)


date_difference_predictors = ["AGE", "NEW_TIME_VARIABLE"]

for predictor in date_difference_predictors:
    if predictor not in cols_pred_num:
        cols_pred_num.append(predictor)
        cols_pred = cols_pred_num + cols_pred_cat
        print(f"predictor {predictor} added to predictor list")

## Brute force interactions

**Finding interactions by brute force. Can work with a limitied number of predictors and/or small dataset only as it is testing each combination of any two predictors meaning its complexity is quadratic and can take very long time with more than lower tens of predictors if the dataset is large.**

The class has two testing methods to find out whether interaction of two variables makes sense. The method is specified in parameter `test_method`:

- *gini*: trains logistic regression model using the two variables and another model using their interaction as the only predictor. Then calculates Gini difference between the model with the interaction and the model using the two original predictors.
- *lr*: performs a likelihood ratio test between two logistic regression models: the richer model is using the two original predictors and their interaction, the simpler model just the two original variables. Calculates p-value of the test

Before this testing is performed, the variables are transformed to be compatible with the logistic regression, which is given by parameters `use_grouping` and `use_grouping_interactions`:

- if `use_grouping` is True, the two original predictors are first grouped and WOE transformed (parameters `groups_per_predictor` and `min_group_size` are taken into account).
- otherwise, the two original predictor are transformed using mean target encoding: numerical predictors are first grouped into `groups_per_predictor` quantiles and then mean target encoding with parameter `mean_target_regularization_weight` and logit transformation is applied. Categorical predictors are mean-target encoded without any pre-grouping, again with parameter `mean_target_regularization_weight`, and the logit transformation is applied.
- if `use_grouping_interactions` is True, interaction is calculated as cartesian product of the transformed original predictors, then each value of the cartesian product is mean target encoded with parameter `mean_target_regularization_weight` and then logit transformation is applied.
- otherwise, interaction is calculated as cartesian product of the transformed original predictors, then grouping and WOE transformation is applied (parameters `groups_per_predictor` and `min_group_size` are taken into account).

**As the result of the test, the pairs with highest Gini differences or lowest p-values should be considered as potential candidates for interaction creation.**

### Gini method

In [ ]:
from scoring.brute_force_interactions import BruteForceInteractions

In [ ]:
bfinter = BruteForceInteractions(
        pred_num=cols_pred_num,
        pred_cat=cols_pred_cat,
        target=col_target,
        base=col_base,
        weight=None,
        test_method="gini",
        use_grouping=False,
        use_grouping_interactions=False,
        groups_per_predictor=5,
        min_group_size=100,
        mean_target_regularization_weight=0.1,
        grouping_category_limit=100,
    )

inter_tab_gini = bfinter.test_interactions(data[train_mask])

In [ ]:
display(inter_tab_gini)

### Likelihood ratio method

In [ ]:
bfinter = BruteForceInteractions(
        pred_num=cols_pred_num,
        pred_cat=cols_pred_cat,
        target=col_target,
        base=col_base,
        weight=None,
        test_method="lr",
        use_grouping=False,
        use_grouping_interactions=False,
        groups_per_predictor=5,
        min_group_size=100,
        mean_target_regularization_weight=0.1,
        grouping_category_limit=100,
    )

inter_tab_lr = bfinter.test_interactions(data[train_mask])

In [ ]:
display(inter_tab_lr)

## Export to Gradient Boosting Workflow

Transformed data can be reused for example in gradient boosting workflow.

You can add more columns to the `metadata_export` dictionary if you plan to use them in Gradient Boosting Workflow (e.g. multiple targets)

In [ ]:
data.to_csv('data_prepared.csv')

In [ ]:
metadata_export["cols_pred_num"] = cols_pred_num
metadata_export["cols_pred_cat"] = cols_pred_cat

json.dump(metadata_export, open("metadata.json", "w", encoding="utf8"), indent=4)

# Grouping and WOE transformation of variables

Don't use such variables which have only 0 or 1 unique levels. Grouping doesn't work for them.

In [ ]:
cols_del = list()
for name, column in tqdm(data[train_mask][cols_pred].iteritems(), total=len(cols_pred)):
    if name in cols_pred_num:
        if column.value_counts(dropna=False).shape[0] < 2:
            cols_del.append(name)
            cols_pred_num.remove(name)
    if name in cols_pred_cat:
        if column.value_counts(dropna=False).shape[0] < 2:
            cols_del.append(name)
            cols_pred_cat.remove(name)

cols_pred = cols_pred_num + cols_pred_cat

if len(cols_del) > 0:
    print("Variables", cols_del, "will not be further used as they have only 1 unique level.")
else:
    print("All predictors have more than 1 unique level.")
del cols_del

In [ ]:
for name, column in tqdm(data[cols_pred_num].iteritems(), total=len(cols_pred_num), leave=False):
    if np.any(np.isinf(column.values)):
        print(f"{name} containes INF values. Please deal with them.")

# data['predictor'].replace(to_replace=np.inf, value=<good idea>, inplace=True)

There are two options how to group your variables. 
1. Automatic grouping groups the variables using a decision tree. User can't change the grouping in any interactive way. The grouping can be saved into external file using its method *save()*. 
2. Interactive grouping is suitable for smaller numbers of variables. User can control which values of each varible will enter which group. The grouping can be saved into external file using the interactive environment.

## Stability grouping

Automatic binning with optimal stable Gini.
    Idea is to make most of the decisions pertaining to design of WOE bins all in one place 
    from point of view of performance and rank time-stability of the resulting bins.
    Aim is to significantly minimize the manual and time consuming interactive binning,
    while at the same time preserving compatibility with the interactive binning should
    that be still occasionally needed (e.g. for business reasons).

**ALGORITHM:**  
for each variable, for n_bins in range:
1. train a tree model (LGBM - see comment below) M, on training dataset, 
    with parameters `min_data_in_leaf = 5%` of training data length
    (this condition can be released for special variables (e.g. hardchecks) -
    in such case the `min_data_in_leaf` is set to 100). 
2. evaluate Gini of M on each time stamp of validation set (most frequently month):
    if we have `k` months, we have `k` ginis. Take average gini of those
    `k ginis --> Gini(n_bins)`
3. At the same time, calculate Elena's Rank Stability Index - RSI (version 1) 
    for target rate : the model M defines a certain grouping of variable
    (e.g. for `n_bins = 3` we have var partitioned into 3 bins) so calculate target rate
    in each bin across the time stamp and  get the RSI as usual. (The RSI for all bins
    is the average of RSI of each constituent bin.  The RSI of a constituent bin is 100%
    if the rank of its target rate relative to other bins's target rates remained 
    constant across all time stamps; otherwise, it decreases depending on how many
    times did the rank of the bin change).

In [ ]:
from scoring.stability_grouping import StableGrouping

sgrouping = StableGrouping(
    columns=cols_pred_num,
    cat_columns=cols_pred_cat,
    bin_stability_threshold=0.10,
    max_leaves=10,
    important_minorities=[],
    must_have_variables=[],
    min_data_in_leaf_for_minotirites=100,
    min_data_in_leaf_share=0.05,
    output_folder='documentation/stability_grouping/',
    show_plots=False,
)

sgrouping.fit(
    X_train=data[train_mask][cols_pred_num+cols_pred_cat],
    X_valid=data[valid_mask][cols_pred_num+cols_pred_cat],
    y_train=data[train_mask][col_target],
    y_valid=data[valid_mask][col_target],
    t_train=data[train_mask][col_month],
    t_valid=data[valid_mask][col_month],
#     w_train=data[train_mask][col_weight],
#     w_valid=data[valid_mask][col_weight],
    progress_bar=True,
)

In [ ]:
sgrouping.save("stability_grouping.json")

## Automatic Grouping
The grouping uses decision tree algorithm and the grouping is supervised based on the target variable. In the following code:

A new instance of **Grouping** class is created. There are two important parameters:
 - *colums*: list of numerical columns to be grouped
 - *cat_columns*: list of categorical columns to be grouped
 - *group_count*: (maximal) number of final groups of each variable
 - *min_samples*: minimal number of observations in each group of each numerical variable
 - *min_samples_cat*: minimal number of observations in each group of each categorical variable

In [ ]:
from scoring.grouping_new.interactive import NewGrouping

grouping = NewGrouping(group_count=5, min_samples=100, min_samples_cat=100, woe_smooth_coef=0.001)

In [ ]:
grouping.fit(
    data=data[train_mask][cols_pred],
    target=data[train_mask][col_target],
    weight=data[train_mask][col_weight],
    progress_bar=True,
    category_limit=10000,
)

Load data from stability grouping.

In [ ]:
# grouping.load(file="stability_grouping.json", show_loaded=True)

## Interactive grouping

New Interactive grouping uses these arguments:
- *data*: training dataset the grouping should be based on
 - *target*: as the grouping is supervised and calculates WOE values, you need to specify the target column name
 - *time_column* time of each observation (usually month) used for stability charts  
 - *weight*: vector of weights of obervation (if not filled, grouping behaves as there are equal weights)

- **Chart section**:
 - For **numerical variables**, there is chart with equifrequncy fine classing (observations as bars, default rate as line), equidistant fine classing and the final groups.
 - For **categorical varibles** there is chart with each of the original categorical values and a chart with the final groups.
 - Stability of default rate and population in time. This uses _time_ column.
- **Variable section**: here you can choose tab with varible which you want to edit. 
 - For **numerical variables**, the tab contains of the borders of the final groups. You can edit these borders, add new with [+] button and remove them with [-] button. You can also manually set WOE for nulls. There is also a button to perform automatic grouping on the selected variable.
 - For **categorical variables**, the tab contains of two tables. In the top table, you can see some statistics for each of the categorical values. In the rightmost column, there is the number of group which is assigned to the category. You can edit this value (doubleclick on it) to change the grouping. In the bottom table you can see statistics for the groups. It is not editable. There is also a button to perform automatic grouping on the selected variable.
 
If this new version does not work for you, the old is still available in scoring library.

In [ ]:
sns.reset_orig()
%matplotlib inline
%config InlineBackend.close_figures=False

grouping.interactive(
    data=data[train_mask][cols_pred],
    target=data[train_mask][col_target],
    time_column=data[train_mask][col_month],
    weight=data[train_mask][col_weight],
)

**Don't forget to *Apply and Save* your changes.**

In [ ]:
#reset the graphical environment to be used by the normal non-interactive charts
sns.set()
%matplotlib inline
%config InlineBackend.close_figures=True

Load the grouping from a file (don't forget to set the right filename) and add the WOE columns to the original dataset.

## Apply the grouping to the dataset

### WOE transformation

Don't forget to apply the grouping to the data. *Grouping.transform()* method now automatically renames columns with proper suffix. If you need to transform just subset of columns use parameter *columns_to_transform=\[...\]*.

In [ ]:
data_woe = grouping.transform(data, transform_to="woe", progress_bar=True)

### Dummy transformation

**Optional:** Transformation to dummy variables. Use if you want to use "full" regression instead of WOE regression

In [ ]:
data_dummy = grouping.transform(data, transform_to='dummy', progress_bar=True)

### Categorical variables transformation

**Optional:** Transformation to categorical variables. Name of categories instead of WOE values - not useful for modelling.

In [ ]:
# data_shortnames = grouping.transform(data, transform_to='shortnames', progress_bar=True)

Plot the fitted WOEs.

In [ ]:
documentation.GroupingPlots(
    data,
    predictors=cols_pred,
    sample="Train",
    target=col_target,
    grouping=grouping.grouping,
    output_folder="documentation",
    #     weight=col_weight
)

## Add the transformed variables to the data set

### WOE variables

Add WOE variabes to the data set.

In [ ]:
woe_columns_to_replace = list()
for column in data_woe.columns:
    if column in data:
        woe_columns_to_replace.append(column)
        print("Column", column, "dropped as it already existed in the data set.")
data = data.drop(woe_columns_to_replace, axis="columns")
data = data.join(data_woe)

del data_woe
gc.collect()

print("Added WOE variables. Number of columns:", data.shape[1])
cols_woe = [s + "_WOE" for s in cols_pred]

### Dummy variables

**Optional.**

In [ ]:
dummy_columns_to_replace = list()
for column in data_dummy.columns:
    if column in data:
        dummy_columns_to_replace.append(column)
        print("Column", column, "dropped as it already existed in the data set.")
data = data.drop(dummy_columns_to_replace, axis="columns")
data = data.join(data_dummy)

del data_dummy
gc.collect()

print("Added dummy variables. Number of columns:", data.shape[1])

#  Feature selection

First remove WOE variables with one WOE value only - they will have no predictive power.

In [ ]:
cols_del = list()
for name, column in data[train_mask][cols_woe].iteritems():
    if (column.count() == 0) or (column.max() == column.min()):
        cols_del.append(name)
        cols_woe.remove(name)

if len(cols_del) > 0:
    print("Variables", cols_del, "will not be further used as they have only 1 unique WOE level.")
else:
    print("All predictors have more than 1 unique WOE level.")

## Predictor power analysis

Calculates IV and Gini of each predictor, sorts the predictors by their power. The power is calculated for each of the samples (train, validate, test, OOT, H.OOT). **If one or more of the samples are empty, comment the according part of the code.**

In [ ]:
from scoring.metrics import iv, gini, lift

power_tab = []
for j in range(0, len(cols_woe)):
    power_tab.append(
        {
            "Name": cols_woe[j],
            "IV Train": iv(data.loc[train_mask, col_target], data.loc[train_mask, cols_woe[j]]),
            "Gini Train": gini(data.loc[train_mask, col_target], -data.loc[train_mask, cols_woe[j]]),
            "IV Validate": iv(data.loc[valid_mask, col_target], data.loc[valid_mask, cols_woe[j]]),
            "Gini Validate": gini(data.loc[valid_mask, col_target], -data.loc[valid_mask, cols_woe[j]]),
            "IV Test": iv(data.loc[test_mask, col_target], data.loc[test_mask, cols_woe[j]]),
            "Gini Test": gini(data.loc[test_mask, col_target], -data.loc[test_mask, cols_woe[j]]),
            "IV OOT": iv(data.loc[oot_mask, col_target], data.loc[oot_mask, cols_woe[j]]),
            "Gini OOT": gini(data.loc[oot_mask, col_target], -data.loc[oot_mask, cols_woe[j]]),
            "IV HOOT": iv(data.loc[hoot_mask, col_target], data.loc[hoot_mask, cols_woe[j]]),
            "Gini HOOT": gini(data.loc[hoot_mask, col_target], -data.loc[hoot_mask, cols_woe[j]]),
        }
    )
power_out = pd.DataFrame.from_records(power_tab)
power_out = power_out.set_index("Name")
power_out = power_out.sort_values("Gini Train", ascending=False)

pd.options.display.max_rows = 1000
display(power_out)
pd.options.display.max_rows = 15
power_out.to_csv(output_folder + "/predictors/covariates.csv")

## Correlations

Show correlation matrix of all the WOE variables

In [ ]:
documentation.Correlations(
    data, predictors=cols_woe, sample="All", output_folder=output_folder + "/analysis/", filename="correlation_full.png"
)

## Hierarchical variable clustering.
- Starts with each variable as a separate cluster
- Creates clusters based on highest average correlations
- The stopping criterion is either parameter *max_cluster_correlation* - once no correlation between clusters is larger than this parameter, the clustering is finished; or *max_clusters* - when this many clusters are created, the clustering is finished. If both specified, the one that makes less clusters is used.

In [ ]:
from scoring import variable_clustering

clustering_correlation = variable_clustering.CorrVarClus(
    max_correlation=0.5,
    # max_clusters=9,
    standardize=True,
    sample_size=50000,
)

clustering_correlation.fit(data[train_mask][cols_woe], data[train_mask][col_target])

Dendrogram (tree scheme of the hierarchical clustering)

In [ ]:
clustering_correlation.draw(output_file=output_folder + "/analysis/clustering_dendrogram.png")

Table of predictors with number of cluster they are in and order in their respective clusters ranked by gini.

In [ ]:
clustering_correlation.display(output_folder + "/predictors/predictor_clusters.csv")

Subset of WOE variables created that contains only the strongest (based on **training Gini**) variable of each cluster. It's up to the user to choose whether they want to used the full set (in this workflow by default) or such restricted set.

In [ ]:
print("Best variables based on correlation clustering:")
clustering_correlation.bestVariables()

There are two other options for clustering in the scoring library:

 - Feature agglomeration (`variable_clustering.FeatureAggVarClus`) which is very similar to correlation. It uses cosine distance (which is closely related to correlation) and you can also specify usage of nearest neighbors connectivity constraints there.
 - k-means with PCA (`variable_clustering.KMeansVarClus`) which uses Euclidean distance between the observations to find the variable clusters. However, Euclidean distance suffers from the curse of dimensionality in high-dimensional spaces, so PCA is applied to the data first. Anyway, this method is not recommended as the dimensionality problem is not solved fully and the results of the previous two methods seems to be much more usable.
 
For reference for these methods, see the classes in scoring library themself (there are docstrings there).

## L1 regularized Logistic Regression

Efficient way how to select subset of predictors from a very big set of covariate. Uses grid search through value of L1 regularization parameter. We start with no predictor in the model and try to add predictors from list called **cols_shortlist** which is defined below (by default, we put there all the WOE variables). The best model selected based on validation Gini.

Interation process can be tuned using various parameters:
 - *steps*: number of steps of grid search
 - *grid_length*: length of the grid for grid search
 - *log_C_init*: initial value of log10 of C parameter for grid search
 - *max_predictors*: maximal number of predictors to enter the model. Ignored if set to 0.
 - *max_correlation*: maximal absolute value of correlation of predictors in the model (variable with larger correlation with existing predictors will not be added to the model)
 - *beta_sgn_criterion*: if this is set to True, all the betas in the model must have the same signature (all positive or all negative)
 - *stop_when_decr*: if this is set to True, the Gini must increase in each iteration. Models with more predictor having lower Gini than the previous will be considered invalid.
 - *stop_immediately*: the iteration process will be stopped immediately after a model which is not fulfilling the criteria (max_predictors, max_correlation, beta_sgn_criterion, stop_when_decr) is found. No further models are searched for.
 - *correlation_sample*: for better performance, correlation matrix is calculated just on a sample of data. The size of the sample is set in this parameter
 - *use_cv*: boolean if Cross Validation should be used instead of train/validation split. In this case, if both training and validation samples are presented to the fit method, they are concatenated together and are used for CV. Gini is evaluated as average of all CV's folds' validation Gini. **Please, be aware that using CV after automatic grouping which was trained using train/validate split might lead to overfitted model.**
 - *cv_folds*: parameter for Cross Validation - number of folds
 - *cv_seed*: parameter for Cross Validation - random seed used to split the folds
 
The *fit* method can be called with two arguments *fit(X,y)* or with four agruments *fit(X_train,y_train,X_valid,y_valid)*. When called with four arguments, the Gini is measured on the validation sample (i.e. validation sample is used for decisions about what steps to be done in stepwise).

There are another optional arguments, *sample_weight* and *sample_weight_valid* where you can put the vector (data set column) with weights of the observations for the train and validation samples.

In [ ]:
#Define a shortlist of predictors to enter the modelling in the next steps.
cols_shortlist = cols_woe
#cols_shortlist = list(set(cols_woe) - set(['unwanted1','unwanted2']))

In [ ]:
from scoring.model_selection import L1GiniModelSelection

modelL1 = L1GiniModelSelection(
    steps=100,
    grid_length=5,
    log_C_init=None,
    max_predictors=200,
    max_correlation=1,
    beta_sgn_criterion=False,
    stop_immediately=False,
    stop_when_decr=False,
    correlation_sample=10000,
    penalty="l1",
    use_cv=False,
    cv_folds=5,
    cv_seed=98765,
    n_jobs=2,
)

modelL1.fit(
    data[train_mask][cols_shortlist],
    data[train_mask][col_target],
    data[valid_mask][cols_shortlist],
    data[valid_mask][col_target],
    #             sample_weight = data[train_mask][col_weight], sample_weight_valid = data[valid_mask][col_weight],
    progress_bar=True,
)

l1predictors = list(modelL1.predictors)

In [ ]:
modelL1.draw_coeff_progression(cols_shortlist, output_folder + "/model/l1path.png")

In [ ]:
modelL1.draw_gini_progression(output_folder + "/model/l1gini.png")

In [ ]:
print("Predictors in the model:", list(modelL1.predictors))

Save the model to disk.

In [ ]:
model_filename1 = "myModelL1.model"
pickle.dump(modelL1, open(model_filename1, "wb"))

Load model.

In [ ]:
# model_filename1 = 'myModelL1'
# modelL1 = pickle.load(open(model_filename1, 'rb'))

The drawback of regularized model is that it is not calibrated, so it must be refitted afterwards. In this workflow, there is stepwise regression after this L1 regression which can serve this purpose (i.e. fitting model with the same set or subset of predictors, but without the regularization).

## Feature selection using LGBM and SHAP values

This sections contains set of methods (functions) that are necessary to develop and fine-tune gradient boosting model.

Note: *lgbm*, *shap* and *hyperopt* packages has to be installed in computer.  
--    `pip install lightgbm shap hyperopt`

In [ ]:
params = {
    "learning_rate": 0.05,
    "num_leaves": 100,
    "colsample_bytree": 0.75,
    "subsample": 0.75,
    "subsample_freq": 1,
    "max_depth": 3,
    "nthreads": 3,
    "verbose": 1,
    "metric": "auc",
    "objective": "binary",
    "early_stopping_rounds": 100,
    "num_boost_round": 100000,
    "seed": 1234,
}

In [ ]:
from scoring import lgbm

model_lgb = lgbm.LGBM_model(cols_pred, params, use_CV=False, CV_folds=3, CV_seed=9876)

### Fit standard or cross-validated model
output: List of lgbm boosters (models)

In [ ]:
model1 = model_lgb.fit_model(
    data[train_mask], data[valid_mask], data[train_mask][col_target], data[valid_mask][col_target]
)

**Predict to unseen dataset**

In case of CV is chosen, then the predictions are average predictions from each of CV models.

In [ ]:
from scoring.metrics import gini

predictions = model_lgb.predict(model1, data[test_mask])
gini(data[test_mask][col_target], predictions)

### Gain or weight variable importances

Output: DataFrame with features and chosen importance

In case of CV is chosen, then the variable importance is computed as the average variable importance from each CV models.


In [ ]:
var_imp = model_lgb.plot_imp(model1, "importance_gain", ret=True, show=True, n_predictors=25)

**Computing shap values for given dataset**

Theoretical background for shap values can be found here https://christophm.github.io/interpretable-ml-book/shapley.html

Output: DataFrame with features and its mean absolute shap values that coresponds with second chart


In [ ]:
var_imp_shap = model_lgb.print_shap_values(
    cols_pred_num,
    cols_pred_cat,
    data[train_mask],
    data[valid_mask],
    data[train_mask][col_target],
    data[valid_mask][col_target],
    data[test_mask],
)

In [ ]:
var_imp_shap

# Scorecard estimator

## Stepwise logistic Regression

### Estimate model

We run stepwise logistic regression on training data set. We start with no predictor in the model and try to add predictors from list called **cols_shortlist2** which is defined below.

We can put there all the WOE variables, but we can also used output of one of the feature selection methods above or some kind of their combination).

In [ ]:
cols_shortlist2 = cols_woe
# cols_shortlist2 = clustering_correlation.bestVariables()
# cols_shortlist2 = modelL1.final_predictors_

Stepwise process can be tuned using various parameters:
 - *initial_predictors*: set of starting predictors (useful for backward method)
 - *max_iter*: maximal number of iterations
 - *min_increase*: minimal marginal Gini contribution for predictor to be added
 - *max_decrease*: minimal marginal Gini diminution for predictor to be removed
 - *max_predictors*: maximal number of predictors to enter the model. Ignored if set to 0.
 - *max_correlation*: maximal absolute value of correlation of predictors in the model (variable with larger correlation with existing predictors will not be added to the model). **This parameter works for "forward" selection method only.**
 - *beta_sgn_criterion*: if this is set to True, all the betas in the model must have the same signature (all positive or all negative). **This parameter works for "forward" selection method only.**
 - *penalty, C*: regularization parameters for logitic regression (sklearn library)
 - *correlation_sample*: for better performance, correlation matrix is calculated just on a sample of data. The size of the sample is set in this parameter
 - *selection_method*: stepwise or forward or backward
 - *use_cv*: boolean if Cross Validation should be used instead of train/validation split. In this case, if both training and validation samples are presented to the fit method, they are concatenated together and are used for CV. Gini is evaluated as average of all CV's folds' validation Gini. **Please, be aware that using CV after automatic grouping which was trained using train/validate split might lead to overfitted model.**
 - *cv_folds*: parameter for Cross Validation - number of folds
 - *cv_seed*: parameter for Cross Validation - random seed used to split the folds
 
The *fit* method can be called with two arguments *fit(X,y)* or with four agruments *fit(X_train,y_train,X_valid,y_valid)*. When called with four arguments, the Gini is measured on the validation sample (i.e. validation sample is used for decisions about what steps to be done in stepwise).

There are another optional arguments, *sample_weight* and *sample_weight_valid* where you can put the vector (data set column) with weights of the observations for the train and validation samples.

In [ ]:
from scoring.model_selection import GiniStepwiseLogit

modelSW = GiniStepwiseLogit(
    initial_predictors=[],
    max_iter=1000,
    min_increase=0.1,
    max_decrease=0.05,
    max_predictors=0,
    max_correlation=0.5,
    beta_sgn_criterion=False,
    penalty="l2",
    C=10e10,
    correlation_sample=10000,
    selection_method="forward",
    use_cv=True,
    cv_folds=5,
    cv_seed=98765,
    n_jobs=2,
)

modelSW.fit(
    data[train_mask][cols_shortlist2],
    data[train_mask][col_target],
    data[valid_mask][cols_shortlist2],
    data[valid_mask][col_target]
    # ,sample_weight = data[train_mask][col_weight], sample_weight_valid = data[valid_mask][col_weight]
)

In [ ]:
pd.options.display.max_rows = 1000
modelSW.print_final_model()

In [ ]:
modelSW.draw_gini_progression(output_folder + "/model/stepwisegini.png")

In [ ]:
modelSW.progress
# to view specific iterations or rows use code below
# modelSW.progress[modelSW.progress['iteration']==1]

### Full Regression

**Optional:** Alternative to WOE logistic regression. Uses dummy variables instead of WOE variables - it is necessary to make Dummy transformation as the output of Grouping instead of WOE variables.

As predictor set which is served to the class *GiniStepwiseFullLogit*, list of original variables (i.e. names of variables before dummy transformation) is used.

Grouping, which was used to create dummy variables, is served to the class as parameter called *dummy_bindings*. The algorithm then automatically extracts bindings between original and dummy variables and uses it during steps of forward/backward/stepwise regression.

The resulting model includes the both the list of final predictors (original variables) and the list of final dummy variables with their coefficients.

In [ ]:
all_predictors = cols_pred

from scoring.model_selection import GiniStepwiseLogit

modelFull = GiniStepwiseLogit(
    initial_predictors=set(),
    all_predictors=set(all_predictors),
    dummy_regression=True,
    dummy_bindings=grouping.get_dummy_names(),
    max_iter=1000,
    min_increase=0.1,
    max_decrease=0.05,
    max_predictors=0,
    selection_method="stepwise",
    use_cv=False,
    cv_folds=5,
    cv_seed=98765,
    n_jobs=2,
)

modelFull.fit(
    data[train_mask],
    data[train_mask][col_target],
    data[valid_mask],
    data[valid_mask][col_target]
    # ,sample_weight = data[train_mask][col_weight], sample_weight_valid = data[valid_mask][col_weight]
)

clf = modelFull

In [ ]:
pd.options.display.max_rows = 1000
modelFull.print_final_model()
pd.options.display.max_rows = 15

### Save model

In [ ]:
model_filename2 = "myModelSW.model"
pickle.dump(modelSW, open(model_filename2, "wb"))

Load model.

In [ ]:
# model_filename2 = 'myModelSW'
# modelSW = pickle.load(open(model_filename2, 'rb'))

In [ ]:
print("Predictors in the model:", list(modelSW.predictors))

## Time stability of predictors

### Stability charts

Set metadata for the stability charts. Two types of charts will be drawn:
- Stability of default rate, for which the variables with default and with base need to be set
- Stability of population, for which the variable with observation count needs to be set

In [ ]:
# data["ID"] = data.index

In [ ]:
clf = modelSW

for col in list(clf.final_predictors_):
    documentation.GroupedEvaluation(
        data,
        predictor=col,
        sample="Observable",
        target=col_target,
        grouping=grouping.grouping,
        weight=col_weight,
        output_folder=output_folder + "/stability",
    )

We should also verify the population stability of the whole time period and stability of shorter target, e.g. FPD30.

In [ ]:
# name of the short target column
target_for_default_short = "FPD"
# name of the short target's base column
base_for_default_short = "FPD_BASE"

In [ ]:
documentation.targets.append((target_for_default_short, base_for_default_short))
documentation.targets

The following code adds base for short target (its name is defined above) if it does not already exist in the data.

In [ ]:
if base_for_default_short not in data:
    data[base_for_default_short] = 0
    data.loc[data[target_for_default_short] == 0, base_for_default_short] = 1
    data.loc[data[target_for_default_short] == 1, base_for_default_short] = 1
    print("Column", base_for_default_short, "added/modified. Number of columns:", data.shape[1])
else:
    print("Column", base_for_default_short, "already exists.")

In [ ]:
documentation.sample_dict["Observable short"] = data[base_for_default_short] == 1
documentation.sample_dict.keys()

In [ ]:
for col in list(clf.final_predictors_):
    documentation.GroupedEvaluation(
        data,
        predictor=col,
        sample="Observable short",
        target=target_for_default_short,
        grouping=grouping,
        weight=col_weight,
        output_folder=output_folder + "/stability_short",
    )

### Stability Index

The following code calculates two versions of Elena's Stability Index, which are defined as follows.

**v1** 
1. Compute the bad rates in each category for each month
2. Compute the rank of each category within each month based on the bad rate
3. Compute the frequency of each position / rank within each category from step 2
4. Compute the ratios of the most frequent position / rank within each category from step 3
5. Compute average of the ratios from step 4

**v2**
1. Compute the bad rates in each category for each month
2. Compute the rank of each category within each month based on the bad rate
3. Compute the frequency of each position / rank through all categories from step 2 and the corresponding ratios
4. Compute the product of the ratios within each position / rank from step 3
5. Compute average of the products from step 4

*Note: Both version can give "false positives" (indicating that variable is unstable) for U-shaped variables.*

In [ ]:
from scoring.stability_index import stability_index_value

stability_tab = []

for pred in clf.final_predictors_:
    for mask in ["train_mask", "valid_mask", "test_mask", "oot_mask", "hoot_mask"]:
        for ver in ["v1", "v2"]:
            stability_tab.append(
                {
                    "Name": pred,
                    "Sample": mask[:-5],
                    "Index version": ver,
                    "Index value": stability_index_value(data[eval(mask)], pred, col_target, col_base, col_month)[ver],
                }
            )

stability_tab = pd.DataFrame(stability_tab)
stability_tab = stability_tab.groupby(["Name", "Index version", "Sample"])[["Index value"]].mean().unstack(level=[1, 2])

pd.options.display.max_rows = 1000
display(stability_tab.round(3))
stability_tab.to_csv(output_folder + "/stability/stability_index.csv")

### PSI

In [ ]:
from scoring.stability_index import psi_calc_df

monthly_psi, masked_psi = psi_calc_df(data, cols_pred_psi=clf.final_predictors_, col_month="MONTH")
display(monthly_psi)

## Model with exact set of predictors

Use predictors from stepwise model for marginal contribution test

### Estimate model

In [ ]:
clf = modelSW
cols_mc_shortlist = list(clf.predictors)

Fit a model with an exact set of predictors. Set *cols_exactlist* to list of your chosen predictors.
 - *cols_exactlist*: list of predictors to fit with a model

In [ ]:
%%capture --no-display
from scoring.model_selection import GiniStepwiseLogit

cols_exactlist = cols_mc_shortlist

modelSW1 = GiniStepwiseLogit(
    initial_predictors=cols_exactlist,
    max_iter=0,
    n_jobs=2,
)
modelSW1.fit(
    data[train_mask][cols_exactlist],
    data[train_mask][col_target],
    data[valid_mask][cols_exactlist],
    data[valid_mask][col_target],
)
m1 = modelSW1.progress.iloc[0]["Gini"]
display(Markdown("Predictors: {}".format(",\n ".join(cols_exactlist))))
display(Markdown("Gini: **{}**".format(m1)))

### Save model

In [ ]:
model_filename3 = "myModelSW1.model"
pickle.dump(modelSW1, open(model_filename3, "wb"))

Load model.

In [ ]:
# model_filename3 = 'myModelSW'
# modelSW1 = pickle.load(open(model_filename3, 'rb'))

## Marginal contribution

Calculate marginal contribution for adding and removing predictors to/from a chosen set.
 - *cols_mc*: base list of predictors
 - *cols_to_add*: list of predictors to be added

In [ ]:
%%capture --no-display
cols_mc_to_add = cols_woe

modelSW1.marginal_contribution(
    data[train_mask],
    data[train_mask][col_target],
    data[valid_mask],
    data[valid_mask][col_target],
    predictors_to_add = cols_mc_to_add,
    output_path = output_folder + "/model/marg_cont.csv"
)

## Cluster Surrogates

Based on clustering which you ran in the previous parts of workflow (either *hierarchical* or *k-means*), clusters of original variables were established. In this part of workflow, you can check what would happen if you swapped any of the predictors with another variable from the same cluster.
This is particulartly useful when you can’t use an originally selected predictor (e.g. there is a business reason why the predictor should not be used).
For this object to work, you need list of predictors (from your model), list of all variables and corresponding list of numbers assigning clusters to these variables. In the data set, you need to have all the variables from the list.

In [ ]:
from scoring.model_selection import VarClusSurrogates

surrogates = VarClusSurrogates(
    clustering_correlation.variables_, clustering_correlation.labels_, modelSW.final_predictors_
)

surrogates.fit(
    data[train_mask][cols_woe], data[train_mask][col_target], data[valid_mask][cols_woe], data[valid_mask][col_target]
)

In [ ]:
surrogates.displaySurrogates(output_folder + "/predictors/cluster_surrogates.csv")

## Score the dataset
**First choose which model is your final model** (into variable *clf*)!

In [ ]:
# clf = modelL1
clf = modelSW1

cols_final_predictors = list(clf.predictors)
pd.DataFrame(cols_final_predictors).to_csv(output_folder + "/predictors/predictors.csv", index=False, header=None)
clf.print_final_model()

Create a new column with the prediction (probability of default).

In [ ]:
col_score = "SCORE"

data[col_score] = clf.predict(data)
print("Column", col_score, "with the prediction added/modified. Number of columns:", data.shape[1])

# Scorecard table output
Output the scorecard to a table. Stats are calculated on a subset of data given by the mask defined below.

In [ ]:
# this mask is an union of masks for training, validation, testing and out of time data sets
table_mask = train_mask | valid_mask | test_mask | oot_mask | hoot_mask

In [ ]:
from scoring.scorecard import ScoreCard

scorecard_new = ScoreCard(
    grouping=grouping.grouping, predictors=clf.predictors, coefficients=clf.coef, intercept=clf.intercept
)

In [ ]:
pd.options.display.max_rows = 1000
display(scorecard_new.scorecard_table_simple().fillna(""))
pd.options.display.max_rows = 15

In [ ]:
pd.options.display.max_rows = 1000
scorecard_out = scorecard_new.scorecard_table_full(data, table_mask, col_target, weightcol=None)
scorecard_out.to_csv(output_folder + "/model/scorecard.csv")
display(scorecard_out.fillna(""))
pd.options.display.max_rows = 15

# Reject inference
In case there were many rejects in the approval process, it might happen that for some values of certain predictors, reject rate was much higher than for others. In this case, so called *„cherry picking“* migh have occured: only the best part of certain sub-population (corresponding to some predictor values) is approved resulting in default rate of this subpopulation being seemingly much lower than it would be normally. This might affect the future performance of the new model, as this sub-population would receive overly optimistic score based on training data.

## Reject inference analysis

In this analysis we see WoE of default rate (based on logarithm of odds „good vs bad“) compared to WoE of reject rate (based on logarithm odds „approve vs reject“). If these two WoEs have opposite trend for a certain predictor, it means that there is potential reject inference issue.

**For this object to work properly, you need to include rejected applications in the data and add a new 0-1 variable containing the rejected flag.** This means that the original masks as *train_mask* can't be used as they consist of observations with measureable defaults only, i.e. there are not the rejected observations in them.

The reject inference analysis can be peformed on two samples (e.g. train and validation) so you can also compare WoEs of these samples against each other.

In [ ]:
from scoring.reject_inference import RejectInferenceCharts

rejectinf = RejectInferenceCharts(target=col_target, reject=col_reject, predictors=cols_final_predictors, weight=None)
rejectinf.fit(data1=data[data[col_datatype] == "train"], data2=data[data[col_datatype] == "valid"])
rejectinf.display()

## Rejected target imputation

Sometimes, we have many rejected cases without observed target, however there exist some kind of proxy target, which can be assigned to these observations so the final model can be scored on them as well as on the usual observations.

For example: there is a high reject rate, but for the rejected cases we have some external score which can tell us to some extent whether they woul default or not. To avoid reject inference we want to add these observations into our trainig sample.

**TargetImputer** is a class for target imputation based on pre-calculated target probabilities. Typical usage is if we want to assign a proxy target to rows where target is unobserved.
There are three imputation types that can be used:
- *randomized*: for each observation where we want to impute the target, an random number between 0 and 1 is generated. Then this ranom number is compared with the pre-calculated probability and target is assigned based on this comparison.
- *cutoff*: for each observation where we want to impute the target, the pre-caluclated probability is compared with this cutoff value and target is assigned based on this comparison.
- *weighted*: for each observation where we want to impute the target, two observations are generated. Each of them has a different value of the target, one of the has weight p and the other has weight 1-p, where p is the pre-calculated target probability.

When we initiate an instance of this class, we use the following arguments:
- imputation_type - Imputation which we want to use, can be 'randomized', 'cutoff' or 'weighted'
- cutoff - Cutoff value which is used if 'cutoff' imputation_type is selected (default: 0.5)
- random_seed - Random seed which is used if 'randomized' imputation_type is selected (default: 987)

Then, *fit* method is called, where the dataset where the target should be imputed is specified. This dataset must contain a column with pre-caluclated target probabilities (floats between 0 and 1) and a column with indicator whether target should be imputed for given row (integers with values 0 or 1). The method then calculates the imputed target using the parameter specified during initialization.

And finally, *transform* method add the imputed values into the dataset. We can specify, whether they should rewrite the original values or be added as new columns. For *weighted* imputation method, it is recommended to use optional argument `reset_index=True`, because this method duplicates some rows, and having rows with duplicate index can cause error in other parts of the workflow.

In [ ]:
# from scoring.reject_inference import TargetImputer

# targetImputer = TargetImputer(imputation_type='randomized', random_seed=987)

# targetImputer.fit(data = data,
#                   col_probs = col_score,
#                   col_reject = col_reject,
#                   #col_weight = col_weight,
#                   prob_of = 1)

# data = targetImputer.transform(data = data,
#                                col_target = col_target,
#                                #col_weight = col_weight,
#                                as_new_columns = True,
#                                reset_index = True)

# Correlations
Calculate and visualise correlation matrix

In [ ]:
documentation.Correlations(
    data,
    predictors=cols_final_predictors,
    sample="All",
    output_folder=output_folder + "/analysis/",
    filename="correlation.png",
)

Show the list of the highest correlation (restricted to correlations that are, in absolute value, higher than *max_ok_correlation* parameter):

In [ ]:
cormat = data[cols_final_predictors].corr()
max_ok_correlation = 0.0

# find highest pairwise correlation (correlation greater than .. in absolute value)
hicors = []
for i in range(0, len(cormat)):
    for j in range(0, len(cormat)):
        if (cormat.iloc[i][j] > max_ok_correlation or cormat.iloc[i][j] < -max_ok_correlation) and i < j:
            hicors.append((i, j, cormat.index[i], cormat.index[j], cormat.iloc[i][j], abs(cormat.iloc[i][j])))
hicors.sort(key=lambda tup: tup[5], reverse=True)

hicors2 = pd.DataFrame(list(zip(*list(zip(*hicors))[2:5])))

# print list of highest correlations
hicors2.head(10)

# Performance characteristics
Performance characteristics of the model (Gini, Lift) and their visualisations.

## Import from Gradient boosting workflow

**If you used the Gradient boosting workflow** to develop the model, here you can import all the metadata from Gradient boosting workflow and continue from here. **PSW will be able to generate all the performance characteristics analysis**, score distribution and also comparison with another score and short target performance.

Otherwise, keep this outcommented.

In [ ]:
# import time
# import datetime
# import operator
# import math
# import random
# import numpy as np
# import pandas as pd

# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os.path
# import pickle
# import gc
# # check your tqdm version if import fails
# from tqdm.notebook import tqdm

# import sys

# sys.path.insert(0, "..")
# import scoring

# metadata = json.load(open("metadata_gb_wfl.json", "r", encoding="utf8"))

# col_time = metadata["col_time"]
# col_month = metadata["col_month"]
# col_day = metadata["col_day"]
# col_target = metadata["col_target"]
# col_base = metadata["col_base"]
# col_weight = metadata["col_weight"]
# col_reject = metadata["col_reject"]
# col_datatype = metadata["col_datatype"]
# col_id = metadata["col_id"]
# col_score = metadata["col_score"]

# from scoring import db
# data = db.read_csv('data_from_gb_wfl.csv', index_col=col_id)
# data[col_id] = data.index

# train_mask = (data[col_datatype] == 'train') & (data[col_base] == 1)
# valid_mask = (data[col_datatype] == 'valid') & (data[col_base] == 1)
# test_mask = (data[col_datatype] == 'test') & (data[col_base] == 1)
# oot_mask = (data[col_datatype] == 'oot') & (data[col_base] == 1)
# hoot_mask = (data[col_datatype] == 'hoot') & (data[col_base] == 1)
# observable_mask = (data[col_base] == 1)

# sns.set()
# %matplotlib inline
# %config InlineBackend.close_figures=True
# from IPython.display import display, Markdown, HTML
# pd.options.display.max_columns = None
# pd.options.display.max_rows = 15
# output_folder = 'documentation_lgbm'

# if not os.path.exists(output_folder): os.makedirs(output_folder)
# if not os.path.exists(output_folder+'/performance'): os.makedirs(output_folder+'/performance')
# if not os.path.exists(output_folder+'/predictors'): os.makedirs(output_folder+'/predictors')
# if not os.path.exists(output_folder+'/stability'): os.makedirs(output_folder+'/stability')
# if not os.path.exists(output_folder+'/stability_short'): os.makedirs(output_folder+'/stability_short')
# if not os.path.exists(output_folder+'/analysis'): os.makedirs(output_folder+'/analysis')
# if not os.path.exists(output_folder+'/model'): os.makedirs(output_folder+'/model')
# if not os.path.exists(output_folder+'/nan_share'): os.makedirs(output_folder+'/nan_share')
    
# from scoring import doctools
# documentation = pickle.load(open("documentation_gb_wfl.pkl", "rb"))

## Performance per sample

If some od these samples (train, valid, test, OOT, HOOT) are empty (i.e. you don't use them), **you need to comment their according rows in some of the following cells.** Such rows are marked by short comments at their ends.

In [ ]:
from scoring.metrics import gini, lift, kolmogorov_smirnov, eval_performance_wrapper

lift_perc = 10

eval_masks = {
    "train": train_mask,
    "valid": valid_mask,
    "test": test_mask,
    "oot": oot_mask,
    "hoot": hoot_mask,
}

In [ ]:
perf = eval_performance_wrapper(
    data=data,
    masks=eval_masks,
    col_target=col_target,
    col_score=col_score,
    # col_weight=col_weight,
    lift_perc=10,
)
display(perf)
perf.to_csv(output_folder + "/performance/performance.csv")

In [ ]:
from scoring.tools import curves_wrapper

curves_wrapper(
    data=data,
    masks=eval_masks,
    col_target=col_target,
    col_score=col_score,
    # col_weight=col_weight,
    lift_perc=10,
    output_folder=output_folder + "/performance/",
)

In [ ]:
# scoring.plot.plot_kolmogorov_smirnov(data[col_target], data[col_score])

## Gini in time

In [ ]:
from sklearn.metrics import roc_curve, auc

def proc_gini(x,y,z):
    fpr, tpr, _ = roc_curve(x[y], x[z], pos_label=0)
    roc_gini = (auc(fpr, tpr)-0.5)*2
    return roc_gini
%matplotlib inline
plt.figure(figsize = (10,7))

len1 = 0

grouped = data[hoot_mask].groupby(col_month, axis=0) #hoot
res_hoot= grouped.apply(proc_gini, col_target ,col_score) #hoot
plt.plot(range(len1,len1+len(res_hoot)),-res_hoot, linewidth=2.0,label='hist. OOT', color = 'm', marker='o') #hoot

if res_hoot is not None: len1 = len1 + len(res_hoot)

grouped = data[train_mask].groupby(col_month, axis=0) #train
res_train= grouped.apply(proc_gini, col_target ,col_score) #train
plt.plot(range(len1,len1+len(res_train)),-res_train, linewidth=2.0,label='Train', color = 'g', marker='o') #train
grouped = data[valid_mask].groupby(col_month, axis=0) #valid
res_valid= grouped.apply(proc_gini, col_target ,col_score) #valid
plt.plot(range(len1,len1+len(res_valid)),-res_valid, linewidth=2.0,label='Validation', color = 'r', marker='o') #valid
grouped = data[test_mask].groupby(col_month, axis=0) #test
res_test= grouped.apply(proc_gini, col_target ,col_score) #test
plt.plot(range(len1,len1+len(res_test)),-res_test, linewidth=2.0,label='Test', color = 'y', marker='o') #test

if res_train is not None: len1 = len1 + len(res_train)

grouped = data[oot_mask].groupby(col_month, axis=0) #oot
res_oot= grouped.apply(proc_gini, col_target ,col_score) #oot
plt.plot(range(len1,len1+len(res_oot)),-res_oot, linewidth=2.0,label='OOT', color = 'b', marker='o') #oot

plt.xticks(range(len(res_train)+len(res_oot)), np.sort(data[col_month].unique()), rotation=45)
plt.xticks(range(len(data[col_month].unique())), np.sort(data[col_month].unique()), rotation=45)


plt.ylim([0,1])
plt.title('Gini by months')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('Months')
plt.ylabel('Gini')
plt.savefig(output_folder+'/performance/ginistability.png', bbox_inches='tight', dpi = 72)
plt.show()

## Performance - bootstrap

In this part, we make bootstrapped sample from the defined dataset `n_iter` times, measure Gini of the score each time and the calculate the (5%, 95%) confidence interval for Gini from it.

In [ ]:
# number of iterations for bootstrapping
n_iter = 100
# confidence interval range in percent (the CI will be calculated as [ci_range, 100-ci_range])
ci_range = 5

from scoring.metrics import bootstrap_gini

bootstrap_results = []

for name, mask in tqdm(eval_masks.items(), leave=False):
    mean, std, ci = bootstrap_gini(
        data=data[mask],
        col_target=col_target,
        col_score=col_score,
        # col_weight=col_weight,
        n_iter=n_iter,
        ci_range=ci_range,
    )
    bootstrap_results.append([name, mean, std, ci[0], ci[1]])

bootstrap_results = pd.DataFrame(bootstrap_results).set_index(0)
bootstrap_results.index.rename("Sample", inplace=True)
bootstrap_results.columns = ["Gini Mean", "Gini std", "CI 5%", "CI 95%"]

In [ ]:
display(bootstrap_results)
bootstrap_results.to_csv(output_folder + "/performance/bootstrap_performance.csv")

# Score distribution

## Distribution charts

In [ ]:
from scoring.plot import plot_score_dist

data["score_lin"] = np.log(data[col_score] / (1 - data[col_score]))
distr_linear = plot_score_dist(
    data[train_mask | valid_mask | test_mask | oot_mask | hoot_mask],
    score_name="score_lin",
    target_name=col_target,
    weight_name=None,
    n_bins=25,
    savefile=output_folder + "/model/distr_linear.png",
)

distr_pd = plot_score_dist(
    data[train_mask | valid_mask | test_mask | oot_mask | hoot_mask],
    score_name=col_score,
    target_name=col_target,
    weight_name=None,
    n_bins=100,
    min_score=0,
    max_score=1,
    savefile=output_folder + "/model/distr_pd.png",
)

## Calibration chart

In [ ]:
documentation.ScoreCalibration(
    data,
    score=col_score,
    sample=["Train", "Valid", "Test", "OOT", "HOOT"],
    target=col_target,
    output_folder=output_folder + "/model/",
)

# Comparison with another score

## Performance comparison
Similar charts to what were already done for the new scorecard are now drawn to compare the new scorecard to another scorecard. The value of the old score should be saved in a special column of original data set.

In [ ]:
col_oldscore = "OLD_SCORE"

# define subset dataOld where old score is not null
old_mask = pd.notnull(data[col_oldscore])

In [ ]:
documentation.sample_dict["Old comparison"] = (test_mask | oot_mask | hoot_mask) & old_mask
documentation.sample_dict.keys()

In [ ]:
eval_old_masks = {
    "valid": valid_mask & old_mask,
    "test": test_mask & old_mask,
    "oot": oot_mask & old_mask,
    "hoot": hoot_mask & old_mask,
}

If the score gives the complementary probability (of non-default), run the following. Otherwise, don't run it.

In [ ]:
data[col_oldscore] = 1 - data[col_oldscore]

In [ ]:
lift_perc = 10

perf_oldscore = eval_performance_wrapper(
    data=data,
    masks=eval_old_masks,
    col_target=col_target,
    col_score=[col_score, col_oldscore],
    # col_weight=col_weight,
    lift_perc=10,
)

display(perf_oldscore)
perf_oldscore.to_csv(output_folder + "/performance/comparison_performance.csv")

In [ ]:
from scoring.tools import curves_wrapper

curves_wrapper(
    data=data,
    masks={"test & oot": (test_mask | oot_mask | hoot_mask) & old_mask},
    col_target=col_target,
    col_score=[col_score, col_oldscore],
    # col_weight=col_weight,
    lift_perc=10,
    output_folder=output_folder + "/performance/comparison_",
)

## Gini in time comparison

In [ ]:
documentation.ScoreComparison(
    data,
    scores=[col_score, col_oldscore],
    sample="Old comparison",
    target=col_target,
    output_folder=output_folder + "/performance",
    filename="comparison_ginistability.png",
)

## Transition matrices

Matrices for the observable population

In [ ]:
from scoring.plot import transmatrix

transmatrix(
    oldscore=data[(valid_mask | test_mask | oot_mask | hoot_mask) & old_mask][col_oldscore],
    newscore=data[(valid_mask | test_mask | oot_mask | hoot_mask) & old_mask][col_score],
    target=data[(valid_mask | test_mask | oot_mask | hoot_mask) & old_mask][col_target],
    base=data[(valid_mask | test_mask | oot_mask | hoot_mask) & old_mask][col_base],
    obs=data[(valid_mask | test_mask | oot_mask | hoot_mask) & old_mask][col_base],
    draw_default_matrix=True,
    draw_transition_matrix=True,
    savepath=output_folder + "/analysis/devpop_",
    quantiles_count=10,
)

Transition matrix for the whole population (put also the rejected etc. here)

In [ ]:
pop_mask = data[col_datatype] != "train"

transmatrix(
    oldscore=data[pop_mask & old_mask][col_oldscore],
    newscore=data[pop_mask & old_mask][col_score],
    target=data[pop_mask & old_mask][col_target],
    base=data[pop_mask & old_mask][col_base],
    obs=data[pop_mask & old_mask][col_base],
    draw_default_matrix=False,
    draw_transition_matrix=True,
    savepath=output_folder + "/analysis/allpop_",
    quantiles_count=10,
)

# Performance on short target
If there is also a shorter (e.g. FPD30) target in the original dataset, we draw also charts for performance on this target in this part of the workflow.

In [ ]:
# name of the short target column
col_short = "FPD"
# name of the short target's base column
col_shortbase = "FPD_BASE"

If you don't have base column in your data set, the following code adds it. **Otherwise, don't run it.**

In [ ]:
if col_shortbase not in data:
    data[col_shortbase] = 0
    data.loc[data[col_short] == 0, col_shortbase] = 1
    data.loc[data[col_short] == 1, col_shortbase] = 1
    print("Column", col_shortbase, "added/modified. Number of columns:", data.shape[1])
else:
    print("Column", col_base, "already exists.")

In [ ]:
shortbase_mask = ((data[col_datatype] == "test") | (data[col_datatype] == "oot") | (data[col_datatype] == "hoot")) & (
    data[col_shortbase] == 1
)

In [ ]:
lift_perc = 10

perf_shorttarget = eval_performance_wrapper(
    data=data,
    masks={"short base test & oot": shortbase_mask},
    col_target=col_target,
    col_score=col_score,
    # col_weight=col_weight,
    lift_perc=10,
)

display(perf_shorttarget)
perf_shorttarget.to_csv(output_folder + "/performance/performance_shorttarget.csv")

In [ ]:
from sklearn.metrics import roc_curve, auc


def proc_gini(x, y, z):
    fpr, tpr, _ = roc_curve(x[y], x[z], pos_label=0)
    roc_gini = (auc(fpr, tpr) - 0.5) * 2
    return roc_gini


%matplotlib inline
plt.figure(figsize=(10, 7))
grouped = data[valid_mask | test_mask | oot_mask | hoot_mask].groupby(col_month, axis=0)
res_new = grouped.apply(proc_gini, col_target, col_score)
plt.plot(range(len(res_new)), -res_new, linewidth=2.0, label="target", color="g", marker="o")

grouped = data[shortbase_mask].groupby(col_month, axis=0)
res_short = grouped.apply(proc_gini, col_short, col_score)
plt.plot(range(len(res_short)), -res_short, linewidth=2.0, label="short target", color="r", marker="o")

plt.xticks(range(len(data[col_month].unique())), np.sort(data[col_month].unique()), rotation=45)

plt.ylim([0, 1])
plt.title("Gini by months")
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.xlabel("Months")
plt.ylabel("Gini")
plt.savefig(output_folder + "/performance/ginistability_shorttarget.png", bbox_inches="tight", dpi=72)
plt.show()

# Scorecard code export
This uses scorecard table generated above (before reject inference and performance analyses).

## SQL code

To run the scorecard on Oracle DWH.

In [ ]:
print(scorecard_new.to_SQL(ntbOut=False, file="scorecard.sql", output_folder=output_folder))

# generate special SQL query with grouping as requested by VN analytical team
# scorecard_new.to_SQL_with_grouping(ntbOut=True, file='scorecard_with_grouping.sql', output_folder=output_folder)

## Blaze table

To be imported using Blaze Remote Desktop *Blaze Tools*.

In [ ]:
scorecard_new.to_blaze_rd(ntbOut=False, file="my_blaze_scorecard.csv", output_folder=output_folder)

# generate slighly different format of Blaze Table compatible with Russian Blaze tools
# scorecard_new.to_blaze(ntbOut=True, file='my_blaze_scorecard.csv', output_folder=output_folder)

## Python code

To run the scorecard in any Python script independently on this workflow.

In [ ]:
print(scorecard_new.to_python(ntbOut=False, file="my_python_scorecard.py", output_folder=output_folder))

# HTML documentation

Create a basic HTML document with important scorecard characteristics. The results of some of the previous parts of the workflow have to be already created on disk, this part will just wrap them up.

If some specific parts (short target analysis, old score comparison) were not done, use the parameters below and set them to *False*.

In [ ]:
txt_scorecard_name = "My PSW model"
txt_author_name = "John Doe"
short_target_analysis_done = True
old_score_comparison_done = True

In [ ]:
with open(output_folder + "/documentation.html", "w", encoding="utf-8") as f:
    f.write("<html>\n<head>\n<title>" + txt_scorecard_name + "</title>\n")
    f.write('<meta charset="utf-8">\n')
    f.write(
        "<style>\nbody{font: normal 10pt Helvetica, Arial, sans-serif;}\n"
        + ".textbold{font-weight:bold;}\n"
        + ".divcode{font-family:Courier New,Courier,Lucida Sans Typewriter,Lucida Typewriter,monospace;}\n"
        + ".divpic{padding-bottom: 20pt;}\n"
        + ".textlabel{font-style:italic;font-size:8pt;}\n"
        + "table{border-collapse:collapse;}\n"
        + "</style>\n"
    )
    f.write("</head>\n<body>")
    f.write("<h1>" + txt_scorecard_name + " - documentation</h1>\n")
    f.write("<h2>Document information</h2>\n")
    f.write('<div class="divpar">\n')
    f.write(' <div class="divtext"><span class="textbold">Author:</span> ' + txt_author_name + "</div>\n")
    f.write(
        ' <div class="divtext"><span class="textbold">Date:</span> '
        + datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        + "</div>\n"
    )
    f.write("</div>\n")
    f.write("<h2>Data sample</h2>\n")
    f.write("<h3>Target</h3>")
    f.write('<div class="divpar">\n')
    f.write(
        ' <div class="divtext"><span class="textbold">Target variable:</span> '
        + pd.read_csv(output_folder + "/model/metadata.csv", header=None, index_col=0).loc["col_target"][1]
        + "</div>\n"
    )
    f.write(
        ' <div class="divtext"><span class="textbold">Base variable:</span> '
        + pd.read_csv(output_folder + "/model/metadata.csv", header=None, index_col=0).loc["col_base"][1]
        + "</div>\n"
    )
    f.write("</div>\n")
    f.write("<h3>Sample characteristics</h3>\n")
    f.write('<div class="divpar">\n')
    f.write(
        ' <div class="divpic"><img src="analysis/data.png" />\n'
        + ' <br /><span class="textlabel">Observations and defaults in time</span></div>\n'
    )
    f.write(
        ' <div class="divtab">\n'
        + pd.read_csv(output_folder + "/analysis/summary.csv", header=[0, 1], index_col=0).to_html(na_rep="")
        + "\n </div>\n"
    )
    f.write("</div>\n")
    f.write("<h3>Covariates</h3>\n")
    f.write('<div class="divpar">\n')
    f.write(
        ' <div class="divtab">\n'
        + pd.read_csv(output_folder + "/predictors/covariates.csv", header=0, index_col=0).to_html(na_rep="")
        + "\n </div>\n"
    )
    f.write("</div>\n")
    f.write("<h2>Final scorecard</h2>\n")
    f.write("<h3>Scorecard</h3>\n")
    f.write('<div class="divpar">\n')
    f.write(
        ' <div class="divtab">\n'
        + pd.read_csv(
            output_folder + "/model/scorecard.csv", header=0, index_col=0, keep_default_na=False, na_values=[""]
        ).to_html(na_rep="")
        + "\n </div>\n"
    )
    f.write("</div>\n")
    f.write("<h3>Scoring SQL</h3>\n")
    f.write('<div class="divpar">\n')
    f.write(
        ' <div class="divcode">\n'
        + open(output_folder + "/model/scorecard.sql", "r").read().replace(" ", "&nbsp;").replace("\n", "<br />")
        + "\n </div>\n"
    )
    f.write("</div>\n")
    f.write("<h2>Predictors</h2>\n")
    for pred in pd.read_csv(output_folder + "/predictors/predictors.csv", index_col=None, header=None)[0].tolist():
        pred0 = "".join(pred.split())[:-4]
        f.write("<h3>" + pred0 + "</h3>\n")
        f.write("<h4>Grouping</h4>")
        f.write('<div class="divpar">\n')
        f.write(' <div class="divpic"><img src="predictors/' + pred0 + '_binning.png" /></div>\n')
        f.write("</div>\n")
        f.write("<h4>Stability</h4>")
        f.write('<div class="divpar">\n')
        f.write(' <div class="divpic"><img src="stability/' + pred + '.png" /></div>\n')
        f.write("</div>\n")
    f.write("<h2>Correlations</h2>\n")
    f.write("<h3>Correlation matrix between WOE variables</h3>\n")
    f.write('<div class="divpar">\n')
    f.write(
        ' <div class="divpic"><img src="analysis/correlation.png" />\n'
        + ' <br /><span class="textlabel">Correlation of WOE variables</span></div>\n'
    )
    f.write("</div>\n")
    f.write("<h2>Model evaluation</h2>\n")
    f.write("<h3>Performance</h3>\n")
    f.write("<h4>General performance</h4>\n")
    f.write('<div class="divpar">\n')
    f.write(
        ' <div class="divtab">\n'
        + pd.read_csv(output_folder + "/performance/performance.csv", header=0, index_col=0).to_html(na_rep="")
        + "\n </div>\n"
    )
    f.write(
        ' <div class="divpic"><img src="performance/roc.png" />\n'
        + ' <br /><span class="textlabel">ROC curve</span></div>\n'
    )
    f.write(
        ' <div class="divpic"><img src="performance/lift.png" />\n'
        + ' <br /><span class="textlabel">Lift curve</span></div>\n'
    )
    f.write("</div>\n")
    f.write("<h4>Performance stability</h4>\n")
    f.write('<div class="divpar">\n')
    f.write(
        ' <div class="divpic"><img src="performance/ginistability.png" />\n'
        + ' <br /><span class="textlabel">Stability of Gini in time</span></div>\n'
    )
    f.write("</div>\n")
    f.write("<h3>Marginal Contribution</h3>\n")
    f.write("<h4>Adding predictors</h4>\n")
    f.write(
        ' <div class="divtab">\n'
        + pd.read_csv(output_folder + "/model/marg_cont_add.csv", header=0, index_col=0).to_html(na_rep="")
        + "\n </div>\n"
    )
    f.write("</div>\n")
    f.write("<h4>Removing predictors</h4>\n")
    f.write(
        ' <div class="divtab">\n'
        + pd.read_csv(output_folder + "/model/marg_cont_rem.csv", header=0, index_col=0).to_html(na_rep="")
        + "\n </div>\n"
    )
    if short_target_analysis_done:
        f.write("<h3>Performance on shorter target</h3>\n")
        f.write("<h4>General performance</h4>\n")
        f.write('<div class="divpar">\n')
        f.write(
            ' <div class="divtab">\n'
            + pd.read_csv(output_folder + "/performance/performance_shorttarget.csv", header=0, index_col=0).to_html(
                na_rep=""
            )
            + "\n </div>\n"
        )
        f.write("</div>\n")
        f.write("<h4>Performance stability</h4>\n")
        f.write('<div class="divpar">\n')
        f.write(
            ' <div class="divpic"><img src="performance/ginistability_shorttarget.png" />\n'
            + ' <br /><span class="textlabel">Stability of Gini in time</span></div>\n'
        )
        f.write("</div>\n")
        f.write("<h3>Calibration</h3>\n")
        f.write('<div class="divpar">\n')
        f.write(
            ' <div class="divpic"><img src="model/calibration.png" />\n'
            + ' <br /><span class="textlabel">Model calibration chart</span></div>\n'
        )
        f.write("</div>\n")
    if old_score_comparison_done:
        f.write("<h2>Comparison with current model</h2>\n")
        f.write("<h3>Performance comparison</h3>\n")
        f.write("<h4>General performance</h4>\n")
        f.write('<div class="divpar">\n')
        f.write(
            ' <div class="divtab">\n'
            + pd.read_csv(
                output_folder + "/performance/comparison_performance.csv", header=[0, 1], index_col=0
            ).to_html(na_rep="")
            + "\n </div>\n"
        )
        f.write(
            ' <div class="divpic"><img src="performance/comparison_roc.png" />\n'
            + ' <br /><span class="textlabel">ROC curve</span></div>\n'
        )
        f.write(
            ' <div class="divpic"><img src="performance/comparison_lift.png" />\n'
            + ' <br /><span class="textlabel">Lift curve</span></div>\n'
        )
        f.write("</div>\n")
        f.write("<h4>Performance stability</h4>\n")
        f.write('<div class="divpar">\n')
        f.write(
            ' <div class="divpic"><img src="performance/comparison_ginistability.png" />\n'
            + ' <br /><span class="textlabel">Stability of Gini in time</span></div>\n'
        )
        f.write("</div>\n")
        f.write("<h3>Transition matrices</h3>\n")
        f.write("<h4>Bad rate matrix</h4>\n")
        f.write('<div class="divpar">\n')
        f.write(
            ' <div class="divpic"><img src="analysis/devpop_matrix_default.png" />\n'
            + ' <br /><span class="textlabel">Default rate matrix</span></div>\n'
        )
        f.write("</div>\n")
        f.write("<h4>Transition matrix - development sample</h4>\n")
        f.write('<div class="divpar">\n')
        f.write(
            ' <div class="divpic"><img src="analysis/devpop_matrix_transition.png" />\n'
            + ' <br /><span class="textlabel">Transition matrix</span></div>\n'
        )
        f.write("</div>\n")
        f.write("<h4>Transition matrix - whole population</h4>\n")
        f.write('<div class="divpar">\n')
        f.write(
            ' <div class="divpic"><img src="analysis/allpop_matrix_transition.png" />\n'
            + ' <br /><span class="textlabel">Transition matrix</span></div>\n'
        )
        f.write("</div>\n")
    f.write("</body></html>")
    display(HTML(f"""<a href="{f.name}" target="_blank">Link to documentation <b>{f.name}</b></a>"""))
    print(
        "Link above might show documentation without pictures due to a bug in Jupyter. In that case please open the file locally."
    )